## Long and Tidy
### Pandas function for returning a column in a tidy 'long' format 

In [61]:
import pandas as pd
import numpy as np

Suppose you have a data set that has comma seperated values in one or more columns, and you need to clean up the data set such that each row has a unique value in the column, without affecting the other columns of the data set.This is useful, for example, if you need to plot or map values in a column. 

This function will seperate each value in the column, strip it of leading/trailing white space, append new row[s] containing each additional value with the information contained in the other columns duplicated, and finally, reindex the data set accordingly,  i.e. return a 'tidy' data set in a 'long' format.

In [62]:
def tidy_long(df, col): 
    tidy_df = df.copy() #make a copy to ensure no changes are made to original df
    column = tidy_df[col].str.split(',', expand=True) # splits the column by delimited strings
    column = column.stack().str.strip() #returns a 'long' series of indexed strings, removing whitespace
    column.index = column.index.droplevel(-1) #drops level to match the original df
    column.name = col # provides a name to join
    #column = pd.DataFrame(column) #converts to df for join
    del tidy_df[col] #deletes original column to avoid conflict 
    tidy_df = tidy_df.join(column, how = 'inner')#returns a clean copy of the df 
    tidy_df.reset_index(drop=True, inplace=True)
    return tidy_df

In [63]:
df = pd.DataFrame({'advocacy': ['truckers','nurses','lawyers'], "pro": ['trump, sanders', 'sanders', 'clinton'], "con": ['rubio','clinton','cruz, trump']})
df

,advocacy,con,pro
0,truckers,rubio,"trump, sanders"
1,nurses,clinton,sanders
2,lawyers,"cruz, trump",clinton


In [64]:
df.shape

(3, 3)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 3 columns):
advocacy    3 non-null object
con         3 non-null object
pro         3 non-null object
dtypes: object(3)
memory usage: 96.0+ bytes


In [65]:
new_df = tidy_long(df, 'con')
new_df

,advocacy,pro,con
0,truckers,"trump, sanders",rubio
1,nurses,sanders,clinton
2,lawyers,clinton,cruz
3,lawyers,clinton,trump


In [66]:
new_df = tidy_long(df, 'pro')
new_df

,advocacy,con,pro
0,truckers,rubio,trump
1,truckers,rubio,sanders
2,nurses,clinton,sanders
3,lawyers,"cruz, trump",clinton


In [67]:
new_df2 = tidy_long(new_df, 'con')
new_df2

,advocacy,pro,con
0,truckers,trump,rubio
1,truckers,sanders,rubio
2,nurses,sanders,clinton
3,lawyers,clinton,cruz
4,lawyers,clinton,trump


If you need to perform a similar operation on multiple columns, you can use the function in a for loop as follows: 

In [68]:
col_list = ['pro', 'con']
data = df
result = ()
for i in col_list: 
    tidy = tidy_long(data, i)
    data = tidy

In [69]:
data

,advocacy,pro,con
0,truckers,trump,rubio
1,truckers,sanders,rubio
2,nurses,sanders,clinton
3,lawyers,clinton,cruz
4,lawyers,clinton,trump


In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 3 columns):
advocacy    5 non-null object
pro         5 non-null object
con         5 non-null object
dtypes: object(3)
memory usage: 160.0+ bytes


### Preserve original index

NOTE: The goal here is to get a new 'tidy' data set. So it's important to note that the index is reset 'in place' accordingly. If you prefer to keep the original index alongside the new, simply reset_index before retruning the new dataframe without including the (drop=True, inplace=True) parameters, as follows: 

In [83]:
def tidy_long_ikeep(df, col): 
    tidy_df = df.copy() #make a copy to ensure no changes are made to original df
    column = tidy_df[col].str.split(',', expand=True) # splits the column by delimited strings
    column = column.stack().str.strip() #returns a 'long' series of indexed strings, removing whitespace
    column.index = column.index.droplevel(-1) #drops level to match the original df
    column.name = col # provides a name to join
    #column = pd.DataFrame(column) #converts to df for join
    del tidy_df[col] #deletes original column to avoid conflict 
    tidy_df = tidy_df.join(column, how = 'inner').reset_index()#returns a clean copy of the df, with the original index preserved
    return tidy_df

In [84]:
new_df = tidy_long_ikeep(df, 'con')
new_df

,index,advocacy,pro,con
0,0,truckers,"trump, sanders",rubio
1,1,nurses,sanders,clinton
2,2,lawyers,clinton,cruz
3,2,lawyers,clinton,trump


Note: this will add index columns with each application, and so it is not preferred. 

In [87]:
col_list = ['pro', 'con']
data = df
result = ()
for i in col_list: 
    tidy = tidy_long_ikeep(data, i)
    data = tidy

In [88]:
data

,level_0,index,advocacy,pro,con
0,0,0,truckers,trump,rubio
1,1,0,truckers,sanders,rubio
2,2,1,nurses,sanders,clinton
3,3,2,lawyers,clinton,cruz
4,3,2,lawyers,clinton,trump
